### Objective
Collect necessary data in my financial analytics system. News API from Alpha Vantage, and FIX API from cTrader are my data sources


In [15]:
import sys
sys.path.append(r'C:\Users\jessi\OneDrive\1 Projects\1-svts')

import data_cleaner

df = (ticker='FOREX:EUR', date_from='20210404T0130', sort='EARLIEST')

print(df)



SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (121415620.py, line 6)